# Makine Çevirisi

Bu notebook'ta Tatoeba'da yapılmış İngilizce-Türkçe çevirileri kullanarak İngilizce-Türkçe çeviri yapabilen bir model geliştireceğiz. Kullanılan veriseti aşağıdadır:

http://www.manythings.org/anki/

Oluşturacağımız model aşağıdaki diyagramdaki gibi olacak. Model iki parçadan oluşuyor. İngilizce cümleleri işleyecek ilk parçaya encoder diyoruz. Türkçe cümleler üretecek ikinci parçaya ise decoder denir. İlk önce encoder gelen cümleyi işler ve bir vektör üretir. Bu vektörü düşünce vektörü olarak düşünebiliriz. Encoder aldığı cümleyi işliyor ve özetini bir vektöre yazıyor. Ardından üretilen vektörü decoder alır ve Türkçe kelimeler üretmeye başlar. Eğer model iyi eğitilmişse üretilen Türkçe cümle verdiğimiz İngilizce cümlenin çevirisi olacaktır.

![Flowchart](diag/nmt-diag.png)

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\Maynard\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Decoder'un kelime üretmeye başlaması için öncesinde bir kelime vermemiz gerekir. Aldığı kelimeden sonra üretime başlayacaktır. Bunun için ise verisetinde bulunmayan 'ssss' kullanacağız. 'ssss' tokenini decoder'a verdiğimizde üretime başlayacaktır. Bir de üretimi sonlandırması için end tokenimiz var. Bunu ise yine verisetinde olmayan 'eeee' olarak belirliyoruz. Decoder bu tokeni ürettiği zaman kelime üretimi sonlanacak.

In [3]:
mark_start = 'ssss '
mark_end = ' eeee'

Encoder ve decoder'a vereceğimiz veriler için liste oluşturup cümleleri bu listeye yazıyoruz. data_src'da İngilizce cümleler data_dest'te Türkçe cümleler bulunacak.

In [4]:
data_src = []
data_dest = []

In [5]:
for line in open('data/tur.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')

    tr_text = mark_start + tr_text + mark_end
    
    data_src.append(en_text)
    data_dest.append(tr_text)

Örnek İngilizce cümle.

In [6]:
data_src[100]

'I drove.'

Ve bu cümlenin verisetindeki Türkçe çevirisi.

In [7]:
data_dest[100]

'ssss Araba sürdüm. eeee'

In [8]:
data_src[200000]

'Can you see anything missing?'

In [9]:
data_dest[200000]

'ssss Eksik bir şey görebiliyor musun? eeee'

Toplam veri sayısı.

In [10]:
len(data_src)

473035

### Tokenleştirme

Keras'ın Tokenizer'ında ihtiyacımız olan birkaç fonksiyon daha var. O yüzden bir wrapper oluşturuyoruz.

In [11]:
class TokenizerWrap(Tokenizer):
    
    def __init__(self, texts, padding, reverse=False, num_words=None):
        Tokenizer.__init__(self, num_words=num_words)

        self.fit_on_texts(texts)

        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'
        else:
            truncating = 'post'

        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token] for token in tokens if token != 0]
        
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text, padding, reverse=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'

        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)

        return tokens

data_src'daki İngilizce cümleleri tokenleştiriyoruz. Padding'de pre kullandığımız için eksik cümlelerde cümlelerin başına sıfır eklenecek. Ayrıca reverse=True ile cümleyi ters çeviriyoruz. Bu sayede encoder'ın gördüğü son kelime ile decoder'ın üreteceği ilk cümle eşleşecektir. 

In [12]:
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=True,
                              num_words=None)

Burada ise Türkçe cümleleri tokenleştiriyoruz. Bu sefer padding'de post kullanıyoruz. Yani eksik cümlelerde cümlenin sonuna sıfır sıfır eklenecek.

In [13]:
tokenizer_dest = TokenizerWrap(texts=data_dest, 
                               padding='post',
                               reverse=False,
                               num_words=None)

Tokenleştirme işlemini gerçekleştiriyoruz. Sonuç olarak bütün İngilizce cümleler 11 tokenden ve büütün Türkçe cümleler ise 10 tokenden oluşuyor. RNN'e verdiğimiz inputların boyutu hep aynı olması gerektiği için her cümleyi eşitledik. Cümlede eksik kelimeler varsa sıfır eklenecek, fazla kelimeler varsa cümle kesilecek.

In [14]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


Örnekte tokenleri görebilirsiniz. Cümle 10 kelimeden kısa olduğu için sonuna üç tane sıfır eklendi. 

In [15]:
tokens_dest[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0,    0])

Aynı cümleyi yazı olarakta görebiliriz. 

In [16]:
tokenizer_dest.tokens_to_string(tokens_dest[200000])

'ssss eksik bir şey görebiliyor musun eeee'

Bu da aynı cümlenin İngilizcesi. 

In [17]:
tokens_src[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39])

In [18]:
tokenizer_src.tokens_to_string(tokens_src[200000])

'missing anything see you can'

Aynı zamanda verisetindeki orijinal halinede bakabiliriz.

In [19]:
data_src[200000]

'Can you see anything missing?'

'ssss' kelimesiyle belirlediğimiz başlangıç tokeni.

In [20]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

1

'eeee' kelimesi ile belirlediğimiz bitiş tokeni.

In [21]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

### Input

Encoder için input.

In [22]:
encoder_input_data = tokens_src

Decoder için input ve output. Output inputun bir kaydırılmış hali olacak.

In [23]:
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]

Input ve output örnekleri.

In [24]:
encoder_input_data[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39])

In [25]:
decoder_input_data[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0])

In [26]:
decoder_output_data[200000]

array([2391,    4,   18, 4127,   48,    2,    0,    0,    0])

String'e çevrilmiş hali.

In [27]:
tokenizer_dest.tokens_to_string(decoder_input_data[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [28]:
tokenizer_dest.tokens_to_string(decoder_output_data[200000])

'eksik bir şey görebiliyor musun eeee'

Bir de toplamda kaç kelimemiz var bakalım.

In [29]:
num_encoder_words = len(tokenizer_src.word_index)
num_decoder_words = len(tokenizer_dest.word_index)

21315 tane İngilizce kelimemiz var.

In [30]:
num_encoder_words

21315

94058 tane de Türkçe kelimemiz var. Türkçe sondan eklemeli bir dil olduğu için İngilizce'ye göre çok daha fazla kelime bulunuyor. Modelimiz için "ev" ve "eve" kelimeleri tamamen farklı iki kelime olacaktır.

In [31]:
num_decoder_words

94058

### Encoder

Kelimeler için 100 uzunluğunda vektörler kullanacağız. Vektörler için Stonford Üniversitesi'nin hazır eğitilmiş vektörleri kullanacağız. Kullandığımız vektörler GLoVe ile Wikipedia üzerinde 6 milyar kelime üzerinde eğitilmiş. Tam ismi "glove.6B.100d.txt".

https://nlp.stanford.edu/projects/glove/

In [32]:
embedding_size = 100

GLoVe ile eğitilmiş vektörleri verisetindeki vektörler ile karşılaştırıyoruz. Eğer verisetimizdeki bir kelimenin GLoVe vektörü bulunmuyorsa vektör rastgele sayılardan oluşacak. Sadece İngilizce için hazır eğitilmiş vektör kullanacağız. Türkçe için vektörler rastgele başlayıp eğitim sırasında eğitilecek.

In [33]:
word2vec = {}
with open('glove.6B.100d.txt', encoding='UTF-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec

In [34]:
embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size))
for word, i in tokenizer_src.word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Embedding matrisimizin boyutu.

In [35]:
embedding_matrix.shape

(21315, 100)

Encoder için input layer.

In [36]:
encoder_input = Input(shape=(None, ), name='encoder_input')

Encoder için Embedding layer. Daha önce topladığımız vektörleri weights parametresine atıyoruz. trainable'a True vererek bu vektörleri eğitileilir yapıyoruz. Modeli eğitirken aynı zamanda embedding layer'da eğitilecek.

In [37]:
encoder_embedding = Embedding(input_dim=num_encoder_words,
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='encoder_embedding')

state_size ile sinir ağımızdaki nöronları belirliyoruz.

In [38]:
state_size = 256

Encoder için GRU layer'larımız. Toplamda 3 katlı bir sinir ağı oluşturuyoruz. Son layer bir vektör döndüreceği için return_sequences False olması gerekiyor.

In [39]:
encoder_gru1 = CuDNNGRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = CuDNNGRU(state_size, name='encoder_gru2', return_sequences=True)
encoder_gru3 = CuDNNGRU(state_size, name='encoder_gru3', return_sequences=False)

Oluşturduğumuz encoder layer'larını birbirine bağlayacak bir fonksiyon yazıyoruz.

In [40]:
def connect_encoder():
    net = encoder_input
    
    net = encoder_embedding(net)

    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)

    encoder_output = net
    
    return encoder_output

Fonksiyonu çağırarak encoder'ı oluşturalım.

In [41]:
encoder_output = connect_encoder()

### Decoder

Decoder için iki tane input layer'ımız olması gerekiyor. Bir input layer ile decoder'ın ürettiği vektörü decoder'a vereceğiz diğer input layer ile ise Türkçe cümleleri decoder'a vereceğiz.

In [42]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')

In [43]:
decoder_input = Input(shape=(None, ), name='decoder_input')

Decoder için embedding. Burada hazır eğitilmiş vektörler kullanmıyoruz. Vektörler rastgele oluşturulacak ve eğitim aşamasında eğitilecek. Word2vec veya GLoVe ile eğitilmiş kelime vektörleri kullanılırsa model daha başarılı olacaktır.

In [44]:
decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

Decoder için GRU layer'larımız. Decoder'da her zaman sequence ürettiğimiz için return_sequences True olmalı.

In [45]:
decoder_gru1 = CuDNNGRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = CuDNNGRU(state_size, name='decoder_gru2', return_sequences=True)
decoder_gru3 = CuDNNGRU(state_size, name='decoder_gru3', return_sequences=True)

Son olarak decoder'da bir dense(fully connected) layer oluşturuyoruz. Bu layer'da üretilecek olan kelimeye karar vereceğiz. Aktivasyona softmax yerine liner dedik. Biraz sonra loss fonksiyonu yazdığımızda softmax işlemi loss fonksiyonunda yapılacak. O yüzden liner vermemiz gerekiyor.

In [46]:
decoder_dense = Dense(num_decoder_words,
                      activation='linear',
                      name='decoder_output')

Decoder layer'larını bağlayacak fonksiyon.

In [47]:
def connect_decoder(initial_state):
    net = decoder_input

    net = decoder_embedding(net)
    
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    decoder_output = decoder_dense(net)
    
    return decoder_output

### Modelleri bağlama

Burada eğitilecek olan modeli oluşturuyoruz. Modeli end to end eğitebilmek için encoder ve decoder'ı birbirine bağlıyoruz.

In [48]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

Şimdi de sadece encoder için bir model oluşturuyoruz. Bu modeli düşünce vektörü üretmek için kullanacağız.

In [49]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])

Bir de sadece decoder için model oluşturuyoruz. Bu modele düşünce vektörü vererek kelime üretmesini sağlayabiliriz.

In [50]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

### Loss fonksiyonu

Loss fonksiyonu için sparse cross entropy kullanıyoruz. 

In [51]:
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

Optimizer için RMSprop kullanıyoruz. Learning rate olara 0.001 kullanıyoruz.

In [52]:
optimizer = RMSprop(lr=1e-3)

In [53]:
decoder_target = tf.placeholder(dtype='int32', shape=(None, None))

### Derleme

Eğitilecek modeli derliyoruz. 

In [54]:
model_train.compile(optimizer=optimizer,
                    loss=sparse_cross_entropy,
                    target_tensors=[decoder_target])

Eğitim tamamlandıktan sonra modeli kaydedebilmek için checpoint oluşturuyoruz.

In [55]:
path_checkpoint = 'checkpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)

Eğer eğitilmiş bir model vaarsa o yüklenecek yoksa sıfırdan eğitime başlanacak.

In [56]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print("Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.")
    print(error)

Modele verilecek inputlar.

In [57]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data}

Modele verilecek output.

In [58]:
y_data = {'decoder_output': decoder_output_data}

### Eğitim

Son olarak modeli eğitiyoruz. 256 batch_size ile 10 epoch eğitim yapılacak.

In [59]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256,
                epochs=10,
                callbacks=[checkpoint])

C:\Users\Maynard\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### Çeviri örnekleri

Eğitim tamamlandıktan sonra bu fonksiyon ile çeviri yapabiliriz.

In [60]:
def translate(input_text, true_output_text=None):
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding='pre')
    
    initial_state = model_encoder.predict(input_tokens)
    
    max_tokens = tokenizer_dest.max_tokens
    
    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)
    
    token_int = token_start
    output_text = ''
    count_tokens = 0

    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data}

        decoder_output = model_decoder.predict(x_data)

        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)
        
        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += " " + sampled_word
        count_tokens += 1

    
    print("Input text:")
    print(input_text)
    print()

    print("Translated text:")
    print(output_text)
    print()

    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()

Öncelikle verisetinde bulunan cümlelerin çeviri örneklerine bakalım. 

In [61]:
translate(input_text=data_src[50000], true_output_text=data_dest[50000])

Input text:
Do you know my name?

Translated text:
 adımı biliyor musun eeee

True output text:
ssss Adımı biliyor musun? eeee



In [75]:
translate(input_text=data_src[500], true_output_text=data_dest[500])

Input text:
I hope so.

Translated text:
 öyle umuyorum eeee

True output text:
ssss Öyle olduğunu umuyorum. eeee



In [76]:
translate(input_text=data_src[40], true_output_text=data_dest[40])

Input text:
I lost.

Translated text:
 kaybettim eeee

True output text:
ssss Kayboldum. eeee



In [77]:
translate(input_text=data_src[300000], true_output_text=data_dest[300000])

Input text:
Tom will certainly try to do that.

Translated text:
 tom kesinlikle bunu yapmaya çalışacak eeee

True output text:
ssss Tom kesinlikle onu yapmaya çalışacak. eeee



In [78]:
translate(input_text=data_src[400000], true_output_text=data_dest[400000])

Input text:
You are telling it second hand, aren't you?

Translated text:
 onu ikinci balık söylüyorsun değil mi eeee

True output text:
ssss Onu dolaylı olarak anlatıyorsun, değil mi? eeee



Daha sonra verisetinde olmayan cümleler ile modeli test edelim.

In [100]:
translate(input_text="This summer I went to Japan")

Input text:
This summer I went to Japan

Translated text:
 bu yaz japonya'ya gittim eeee



In [101]:
translate(input_text="Where did you buy this dress?")

Input text:
Where did you buy this dress?

Translated text:
 bu elbiseyi nereden aldın eeee



In [105]:
translate(input_text="Which road leads to the airport?")

Input text:
Which road leads to the airport?

Translated text:
 hangi yol havaalanına gider eeee



### Sonuç:

Sonuç olarak basit cümleleri başarılı bir şekilde çevirebilen bir model elde ettik. Hem modelin küçük olmasından hem de verisetinin yetersiz olmasından dolayı model biraz daha karmaşık cümleleri çevirmekte zorluk yaşıyor. 